<h2 >Sentiment Analysis</h2>

In [1]:
# pip install tweepy

     |████████████████████████████████| 94 kB 2.3 MB/s eta 0:00:011
     |████████████████████████████████| 62 kB 4.5 MB/s  eta 0:00:01
     |████████████████████████████████| 151 kB 12.9 MB/s eta 0:00:01
ERROR: conda 4.14.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: oauthlib
    Found existing installation: oauthlib 3.1.0
    Uninstalling oauthlib-3.1.0:
      Successfully uninstalled oauthlib-3.1.0
Note: you may need to restart the kernel to use updated packages.


In [19]:
import tweepy
import json
import time
import datetime
from datetime import date, timedelta

### Task 1: Data Extraction from Twitter API
---

In [15]:
api_key = '1D9l2cpV4EJaO1jBrdWU9zfoU' 
api_key_secret = 'qxyDTwZUABqd0sjaLe1Q3wQSxBhTjRexWbxDptu34fWt51ohpI'
access_token = '1572868505137188864-JSJF2tFizDiSvvMTUH5UhGjd1wOn5j'
access_token_secret = '6A562B7011LSkzmQs4TNZMLPO3USIw7mh5p89LlyKhOfD'

In [16]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [16]:
#api.search_tweets()

In [194]:
list_of_dicts = []
search_term = 'covid19 covid vaccine'
num_tweets = 11000

In [253]:
list_of_dates = []
today = date.today()
for i in range(-477,-470):
    target_date = (today + timedelta(days=i)).strftime("%Y-%m-%d")
    list_of_dates.append(target_date)

In [254]:
list_of_dates

['2021-06-03',
 '2021-06-04',
 '2021-06-05',
 '2021-06-06',
 '2021-06-07',
 '2021-06-08',
 '2021-06-09']

In [92]:
# def get_tweets(search_term = search_term, num_tweets = num_tweets):

#     with open("sample.json", "w") as outfile:
#         for end_date in list_of_dates:
#             start_date = (datetime.datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=1)).strftime("%Y-%m-%d") # Create 1-day windows for extraction
#             tweet_count = len(list_of_dicts)

# #             tweetCursor = tweepy.Cursor(api.search_tweets,
# #                                        q=f'{search_term} since:{start_date} until:{end_date}',
# #                                         #q=search_term,
# #                                        lang = 'en',
# #                                        count = num_tweets,
# #                                        tweet_mode = 'extended').items(num_tweets)

#             for tweet in tweepy.Cursor(api.search_tweets,
#                                        q=f'{search_term} since:{start_date} until:{end_date}',
#                                       # q=search_term,
#                                        lang = 'en',
#                                        count = num_tweets,
#                                        tweet_mode = 'extended').items(num_tweets):
#                 #if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
#                 #if 'RT @' not in tweet.full_text:
#                     if tweet.lang == "en":
#                         tweet_dict = {}
#                         tweet_dict['username'] = tweet.user.name
#                         tweet_dict['location'] = tweet.user.location
#                         tweet_dict['text'] = tweet.full_text 
#                         tweet_dict['hashtags'] = tweet.entities['hashtags']
#                         dtx = tweet.created_at
#                         tweet_dict['tweet_date'] = dtx.strftime("%m/%d/%Y, %H:%M:%S")
#                         tweet_dict["retweeted"] = tweet.retweeted
#                         tweet_dict["retweet_count"] =  tweet.retweet_count
#                         tweet_dict["favorite_count"] = tweet.favorite_count
#                         #list_of_dicts.append(tweet_dict)
#                         json.dump(tweet_dict, outfile)
#                         if tweet_count % 10 == 0:
#                             print(f'Extracted tweet count = {tweet_count}')
#                         tweet_count +=1
#                         print(tweet_count)
                        
#             #print(f'Completed extraction for {start_date} to {end_date}. Sleep for 15 mins')
#             #time.sleep(900)
#             #print('Ready to go again')

In [246]:
def get_tweets(search_term = search_term, num_tweets = num_tweets):
#     try:
        for end_date in list_of_dates:
            start_date = (datetime.datetime.strptime(end_date, '%Y-%m-%d') - timedelta(days=1)).strftime("%Y-%m-%d") # Create 1-day windows for extraction
            tweet_count = len(list_of_dicts)

            for tweet in tweepy.Cursor(api.search_tweets,
                                       q=f'{search_term} since:{start_date} until:{end_date}',
                                      # q=search_term,
                                       lang = 'en',
                                       count = num_tweets,
                                       tweet_mode = 'extended').items(num_tweets):
                #if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                if 'RT @' not in tweet.full_text:
                    if tweet.lang == "en":
                        tweet_dict = {}
                        tweet_dict['username'] = tweet.user.name
                        tweet_dict['location'] = tweet.user.location
                        tweet_dict['text'] = tweet.full_text 
                        tweet_dict['hashtags'] = tweet.entities['hashtags']
                        dtx = tweet.created_at
                        tweet_dict['tweet_date'] = dtx.strftime("%m/%d/%Y, %H:%M:%S")
                        tweet_dict["retweeted"] = tweet.retweeted
                        tweet_dict["retweet_count"] =  tweet.retweet_count
                        tweet_dict["favorite_count"] = tweet.favorite_count
                        list_of_dicts.append(tweet_dict)
                        #json.dump(tweet_dict, outfile)
                        if tweet_count % 500 == 0:
                            print(f'Extracted tweet count = {tweet_count}')
                        tweet_count +=1
                        #print(tweet_count)
                        
            print(f'Completed extraction for {start_date} to {end_date}. Sleep for 15 mins')
            #time.sleep(900)
            print('Ready again')
#     except tweepy.TweepError:
#             print(tweepy.TweepError)
#             time.sleep(15 * 60)
#         

In [255]:
get_tweets()

Completed extraction for 2021-06-02 to 2021-06-03. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-03 to 2021-06-04. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-04 to 2021-06-05. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-05 to 2021-06-06. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-06 to 2021-06-07. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-07 to 2021-06-08. Sleep for 15 mins
Ready again
Completed extraction for 2021-06-08 to 2021-06-09. Sleep for 15 mins
Ready again


In [235]:
len(list_of_dicts)

1377

In [201]:
import pandas as pd

df = pd.DataFrame(list_of_dicts)
df.head()

,username,location,text,hashtags,tweet_date,retweeted,retweet_count,favorite_count
0,Jerry Rider,QLD Australia,"More evidence of Covid-19 vaccine shedding, lo...","[{'text': 'COVID19', 'indices': [115, 123]}, {...","09/14/2022, 23:32:18",False,1,1
1,Northern Qld PHN,"Cairns, Queensland",💪💉MACKAY COVID-19 VAX CLINIC💉💪 | A free COVID-...,"[{'text': 'Mackay', 'indices': [179, 186]}, {'...","09/14/2022, 23:30:05",False,1,2
2,Jonathan Bradley,Canada,“The reported incident counts reflect #prelimi...,"[{'text': 'preliminaryinformation', 'indices':...","09/14/2022, 23:29:42",False,0,0
3,Northern Qld PHN,"Cairns, Queensland",💪💉TOWNSVILLE COVID-19 VAX CLINIC 💉💪 | A free C...,"[{'text': 'Townsville', 'indices': [188, 199]}...","09/14/2022, 23:24:16",False,1,2
4,Patrick,,"@TomiLahren Same, never been vaxxed for ""covid...","[{'text': 'CovidBS', 'indices': [185, 193]}, {...","09/14/2022, 23:22:12",False,0,2


In [202]:
s_set = {}

for idx in range(0, len(df)):
    s = df["tweet_date"][idx].split(",")[0]
    #print(type(s))
    if s in s_set.keys():
        s_set[s] += 1
    else:
        s_set[s] = 1
    
s_set

{'09/14/2022': 205,
 '09/15/2022': 177,
 '09/16/2022': 181,
 '09/17/2022': 134,
 '09/18/2022': 152,
 '09/19/2022': 166,
 '09/20/2022': 176,
 '09/21/2022': 186}

In [214]:
len(df)

1377

In [230]:
df.to_csv("testdata.csv")